In [3]:
#!pip install pyarrow
#!pip install tensorflow

     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     ---------------------------------------- 57.5/57.5 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/65.5 kB ? eta -:--:--
     ---------------------------------------- 65.5/65.5 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ----------- ---------------------------- 0.4/1.5 MB 13.2 MB/s eta 0:00:01
     ---------------------------- ----------- 1.1/1.5 MB 13.4 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 11.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/181.3 kB ? eta -:--:--
     ------------------------------------- 181.3/181.3 kB 10.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/151.7 kB ? eta -:--:--
     ----------------------------------- -- 143.4/151.7 kB 8.3 MB/s eta 0:00:01
     -------------------------------------- 151.7/151.7 kB 3.0 MB/s eta 0:00:00
  

   ---------------------------------------- 0.0/442.0 kB ? eta -:--:--
   --------------------------------------  440.3/442.0 kB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 442.0/442.0 kB 6.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/183.3 kB ? eta -:--:--
   ---------------------------------------- 183.3/183.3 kB 3.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/102.2 kB ? eta -:--:--
   ------------------------------------ --- 92.2/102.2 kB ? eta -:--:--
   ---------------------------------------- 102.2/102.2 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/226.7 kB ? eta -:--:--
   ---------------------------------------- 226.7/226.7 kB 7.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/84.9 kB ? eta -:--:--
   ---------------------------------------- 84.9/84.9 kB 1.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset,TensorDataset
import pandas as pd
import pyarrow
from torchvision import models, transforms
from sklearn.model_selection import train_test_split

In [7]:
#CONSTANTS
exercises_dict = {
    'Abduction': 0,
    'Bird': 1,
    'Bridge': 2,
    'Knee': 3,
    'Shoulder': 4,
    'Squat' : 5,
    'Stretch' : 6
}

In [16]:
dataset = pd.read_parquet('dataset.parquet', engine='pyarrow')

In [20]:
csv_file_path = 'C:\\Users\\41794\\Documents\\ML\\mlproject2\\ml-project-2-sms-project\\sample_dataframe.csv'

# Write the DataFrame to a CSV file
dataset.to_csv(csv_file_path, index=False)

In [88]:
dataset_side_low = dataset[dataset['Camera']=='Frontal_Low'].copy()

In [89]:
dataset_side_low.head(20)

,Participant,Exercise,Set,Camera,time(s),left_ankle_x,left_ankle_y,left_ankle_z,left_ear_x,left_ear_y,...,right_pinky_z,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_thumb_x,right_thumb_y,right_thumb_z,right_wrist_x,right_wrist_y,right_wrist_z
0,P04,Abduction,Correct,Frontal_Low,0.000000,-0.008330,-0.588275,0.543553,-0.059687,0.563000,...,-0.631847,-0.027612,0.458087,-0.420655,-0.041205,0.957030,-0.592280,-0.044813,0.942533,-0.594150
1,P04,Abduction,Correct,Frontal_Low,0.033333,-0.008765,-0.590462,0.550581,-0.056477,0.563068,...,-0.705299,-0.028432,0.457900,-0.423891,-0.049572,0.951706,-0.657239,-0.052007,0.937681,-0.658462
2,P04,Abduction,Correct,Frontal_Low,0.066667,-0.006981,-0.583870,0.560394,-0.059841,0.562750,...,-0.677595,-0.029204,0.458030,-0.426226,-0.049634,0.954498,-0.634484,-0.052071,0.940601,-0.635520
3,P04,Abduction,Correct,Frontal_Low,0.100000,-0.007052,-0.584022,0.562158,-0.055153,0.561507,...,-0.680671,-0.029263,0.457787,-0.434338,-0.052394,0.959484,-0.636930,-0.054515,0.945177,-0.637935
4,P04,Abduction,Correct,Frontal_Low,0.133333,-0.007104,-0.585524,0.559293,-0.055709,0.557782,...,-0.688323,-0.029205,0.450923,-0.444480,-0.052349,0.948858,-0.643176,-0.054549,0.934714,-0.645006
5,P04,Abduction,Correct,Frontal_Low,0.166667,0.009992,-0.571564,0.567051,-0.051310,0.557735,...,-0.699903,-0.029240,0.450929,-0.446098,-0.056141,0.949609,-0.653387,-0.058393,0.935206,-0.655316
6,P04,Abduction,Correct,Frontal_Low,0.200000,0.009371,-0.567833,0.575095,-0.055348,0.559604,...,-0.666455,-0.029450,0.451026,-0.443491,-0.052681,0.949968,-0.623849,-0.056442,0.935289,-0.624934
7,P04,Abduction,Correct,Frontal_Low,0.233333,0.008817,-0.519276,0.590968,-0.052002,0.559886,...,-0.660430,-0.029533,0.450456,-0.443944,-0.055746,0.952216,-0.618401,-0.058725,0.936812,-0.619375
8,P04,Abduction,Correct,Frontal_Low,0.266667,-0.024134,-0.567463,0.529420,-0.052049,0.568976,...,-0.659184,-0.027994,0.464474,-0.430074,-0.036725,0.952213,-0.616706,-0.039719,0.936779,-0.618030
9,P04,Abduction,Correct,Frontal_Low,0.300000,-0.022681,-0.500166,0.577194,-0.052115,0.569891,...,-0.665598,-0.027203,0.465153,-0.430184,-0.032225,0.950447,-0.622263,-0.034341,0.934459,-0.623324


In [17]:
display(dataset)

,Participant,Exercise,Set,Camera,time(s),left_ankle_x,left_ankle_y,left_ankle_z,left_ear_x,left_ear_y,...,right_pinky_z,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_thumb_x,right_thumb_y,right_thumb_z,right_wrist_x,right_wrist_y,right_wrist_z
0,P04,Abduction,Correct,Frontal_Top,0.000000,0.009328,0.779305,0.187880,0.020658,-0.693419,...,-0.633111,-0.054347,-0.511760,-0.185118,-0.068159,-0.602015,-0.598093,-0.066565,-0.600690,-0.592768
1,P04,Abduction,Correct,Frontal_Top,0.033333,0.009353,0.779298,0.185050,0.021187,-0.693361,...,-0.630835,-0.054293,-0.511703,-0.184861,-0.068130,-0.602200,-0.596219,-0.066508,-0.600835,-0.590916
2,P04,Abduction,Correct,Frontal_Top,0.066667,0.009722,0.779311,0.179212,0.021351,-0.693388,...,-0.630923,-0.054289,-0.511696,-0.184849,-0.067241,-0.601852,-0.596222,-0.065676,-0.600436,-0.590910
3,P04,Abduction,Correct,Frontal_Top,0.100000,0.011696,0.778027,0.166566,0.023431,-0.693102,...,-0.631444,-0.053769,-0.511612,-0.183764,-0.062897,-0.600404,-0.596646,-0.061278,-0.599290,-0.591304
4,P04,Abduction,Correct,Frontal_Top,0.133333,0.011671,0.778045,0.166878,0.023658,-0.693346,...,-0.631149,-0.053763,-0.512331,-0.183866,-0.061787,-0.600463,-0.596128,-0.060647,-0.599425,-0.590991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,P28,Stretch,E,Side_Low,24.333333,0.104921,0.764555,0.082647,0.058097,-0.669222,...,-0.153244,-0.163026,-0.512648,-0.044837,-0.626770,-0.533090,-0.155961,-0.617913,-0.532957,-0.135954
731,P28,Stretch,E,Side_Low,24.366667,0.104855,0.764961,0.084220,0.058165,-0.668928,...,-0.157576,-0.163073,-0.512568,-0.046608,-0.626767,-0.531939,-0.159559,-0.617969,-0.531721,-0.139373
732,P28,Stretch,E,Side_Low,24.400000,0.105392,0.765160,0.084043,0.058210,-0.668465,...,-0.158064,-0.163069,-0.512303,-0.046589,-0.626679,-0.530991,-0.159817,-0.617878,-0.530680,-0.139622
733,P28,Stretch,E,Side_Low,24.433333,0.106408,0.765264,0.084009,0.059145,-0.668389,...,-0.158204,-0.162886,-0.512327,-0.046626,-0.625844,-0.529880,-0.159893,-0.617153,-0.529503,-0.139712


In [29]:
nan_dataset = dataset[dataset.isna().any(axis=1)].copy()

In [30]:
display(nan_dataset)

,Participant,Exercise,Set,Camera,time(s),left_ankle_x,left_ankle_y,left_ankle_z,left_ear_x,left_ear_y,...,right_pinky_z,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_thumb_x,right_thumb_y,right_thumb_z,right_wrist_x,right_wrist_y,right_wrist_z
35,P04,Abduction,Correct,Frontal_Low,1.166667,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572,P04,Abduction,Correct,Side_Low,19.066667,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
573,P04,Abduction,Correct,Side_Low,19.100000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
602,P04,Abduction,Correct,Side_Low,20.066667,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
739,P04,Abduction,Correct,Side_Low,24.633333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1510,P28,Bridge,B,Side_Low,50.333333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1511,P28,Bridge,B,Side_Low,50.366667,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1514,P28,Bridge,B,Side_Low,50.466667,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1515,P28,Bridge,B,Side_Low,50.500000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
dataset.shape

(2210672, 104)

In [31]:
nan_dataset2 = dataset[dataset.isna()].copy()
display(nan_dataset2)

,Participant,Exercise,Set,Camera,time(s),left_ankle_x,left_ankle_y,left_ankle_z,left_ear_x,left_ear_y,...,right_pinky_z,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_thumb_x,right_thumb_y,right_thumb_z,right_wrist_x,right_wrist_y,right_wrist_z
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
#dataset = dataset.dropna()
#size = 100000
#dataset = dataset[:size]
data = dataset.values
indices = dataset.index
labels = dataset.columns

x_train = data[:,4:]
y_train = np.array([exercises_dict[e] for e in data[:,1]])#[::4])

In [67]:
x_train = x_train.astype('float32')
y_train = y_train.astype(np.int64)

In [57]:
dataset.shape

(2210672, 104)

In [58]:
#example of a row with labels
print(np.stack((labels.to_numpy(),data[4 * 1793 + 1]), axis = 1))

[['Participant' 'P04']
 ['Exercise' 'Abduction']
 ['Set' 'Correct']
 ['Camera' 'Side_Low']
 ['time(s)' 59.7]
 ['left_ankle_x' 0.152426078915596]
 ['left_ankle_y' 0.7627389430999756]
 ['left_ankle_z' 0.06026823818683624]
 ['left_ear_x' 0.02042476087808609]
 ['left_ear_y' -0.6602293252944946]
 ['left_ear_z' -0.16169288754463196]
 ['left_elbow_x' 0.38551852107048035]
 ['left_elbow_y' -0.5582109689712524]
 ['left_elbow_z' -0.10423755645751953]
 ['left_eye_center_x' -0.040199458599090576]
 ['left_eye_center_y' -0.6730453372001648]
 ['left_eye_center_z' -0.25773122906684875]
 ['left_eye_inner_x' -0.04069114103913307]
 ['left_eye_inner_y' -0.6725935339927673]
 ['left_eye_inner_z' -0.25846073031425476]
 ['left_eye_outer_x' -0.040479350835084915]
 ['left_eye_outer_y' -0.6734108328819275]
 ['left_eye_outer_z' -0.2576296329498291]
 ['left_foot_x' 0.1924462765455246]
 ['left_foot_y' 0.8504915833473206]
 ['left_foot_z' -0.07770746201276779]
 ['left_heel_x' 0.16198818385601044]
 ['left_heel_y' 0.806

In [68]:
#Splits the dataset in a 4-channel data set (think of an RGB Image) coresponding to the 4 different cameras

lengths = np.append(np.where(indices.array[:len(indices)] == 0)[0], [len(indices)])
ft_x, fl_x, st_x, sl_x = [], [], [], []
mod4 = 0
gap = 0

for idx, l in enumerate(lengths):
    if idx < len(lengths)-1:
        end = lengths[idx + 1]
        
        #if mod4 == 0:
        gap = end - l

        segment = x_train[l:l + gap]
        
        if mod4 == 0:
            ft_x.append(segment)
        elif mod4 == 1:
            fl_x.append(segment)

        elif mod4 == 2:
            st_x.append(segment)

        elif mod4 == 3:
            sl_x.append(segment)
        
        mod4 = (mod4 + 1) % 4

print("STACKING:")
# Concatenate the lists into arrays
ft_x = np.vstack(ft_x) 
fl_x = np.vstack(fl_x)
st_x = np.vstack(st_x) 
sl_x = np.vstack(sl_x)

print(ft_x.shape)
print(fl_x.shape)
print(st_x.shape)
print(sl_x.shape)

STACKING:
(552668, 100)
(552668, 100)
(552668, 100)
(552668, 100)


In [69]:
x_train = np.array([ft_x, fl_x, st_x, sl_x])

In [70]:
x_train = np.moveaxis(x_train,-1,0)
x_train = np.moveaxis(x_train,-1,0)
print(x_train.shape)
print(y_train.shape)

(552668, 100, 4)
(2210672,)


In [71]:
get_mlp = lambda: torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(400, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 7),
)

get_cnn = lambda: torch.nn.Sequential(
    torch.nn.Conv1d(in_channels=4, out_channels=32, kernel_size=5, stride=1, padding=2),
    torch.nn.ReLU(),
    torch.nn.Conv1d(32, 64, 5, stride=2, padding=2),
    torch.nn.ReLU(),
    torch.nn.Conv1d(64, 64, 5, stride=1, padding=2),
    torch.nn.ReLU(),
    torch.nn.Conv1d(64, 128, 5, stride=2, padding=2),
    torch.nn.ReLU(),
    torch.nn.AdaptiveAvgPool1d(1),
    torch.nn.Conv1d(128, 7, 1),
    torch.nn.Flatten(),
)

In [72]:

"""
Return a modified dataset cls that can insert MNIST like images into larger
frames with an option for random shifts, scrambling the images in a
consistent way (using the same shuffling for all images) and adding random
Gaussian noise (to the base data, noise is always the same for a given
image).
"""

class CustomDataset(Dataset):
    def __init__(self, x_train, y_train, transform=None):
        self.x_train = x_train
        self.y_train = y_train
        self.transform = transform

    def __len__(self):
        return len(self.x_train)

    def __getitem__(self, idx):
        sample = {'input': self.x_train[idx], 'label': self.y_train[idx]}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [73]:
input_size = 400  # Number of input channels
hidden_size = 64  # Size of the hidden layer
output_size = 7  # Number of classes (0 to 6)
learning_rate = 1e-3
num_epochs = 10

# Create an instance of the custom dataset
custom_dataset = CustomDataset(x_train, y_train)

# Create a DataLoader
batch_size = 64
custom_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

model = get_mlp()
model_path = 'MLP.path'
model.load_state_dict(torch.load(model_path))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [74]:
#def train_epoch()
for epoch in range(num_epochs):
    losses = []
    for batch in custom_dataloader:
        inputs, labels = batch['input'], batch['label']

        # Flatten the inputs (assuming the data is in the shape (batch_size, 100, 4))
        inputs = inputs.view(-1, input_size)
        # Forward pass
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        losses.append(loss.item())
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {np.mean(losses):.4f}')

Epoch [1/10], Loss: nan


KeyboardInterrupt: 

In [53]:
torch.save(model.state_dict(),'MLP.path')

In [14]:
x_train = np.moveaxis(x_train,-1,0)
print(x_train.shape)


(100, 4, 552545)


In [15]:
input_size = 400  # Number of input channels
hidden_size = 64  # Size of the hidden layer
output_size = 7  # Number of classes (0 to 6)
learning_rate = 1e-3
num_epochs = 10

# Create an instance of the custom dataset
custom_dataset = CustomDataset(x_train, y_train)

# Create a DataLoader
batch_size = 64
custom_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

model = get_cnn()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    losses = []
    for batch in custom_dataloader:
        inputs, labels = batch['input'], batch['label']

        # Forward pass
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        print(loss)
        losses.append(loss.item())
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {np.mean(losses):.4f}')